In [2]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

# Give yourself access to common
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# Gather training and testing data
from common import get_baseline_data, get_nn_features
from sklearn.model_selection import train_test_split

nba = get_baseline_data('../data/cleaned_data/base_cleaned.csv')
X = nba[get_nn_features()].values
y = nba[['inflationAdjSalary']].values
# print(y.max())
# y = y/y.max() # removing the normalization seems to be having a large negative effect
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)



b


c:\Users\mmart\Programming\EECS545\project\EECS-545---Predicting-NBA-Player-s-Salary\common.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return nba


In [52]:
# Fit the model
from sklearn.neural_network import MLPRegressor
import numpy as np

# MLPClassifier only classifies data as integers or strings, therefore, our problem is one of regression for the neural network
# Consequently I should use the mlp regressor
mlp = MLPRegressor(hidden_layer_sizes=(10,10,10), solver='adam', max_iter=1000)
mlp.fit(X_train,y_train)


c:\Users\mmart\Programming\EECS545\project\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\mmart\Programming\EECS545\project\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(10, 10, 10), max_iter=1000)

In [53]:
# Tune the Hyerparameters
from sklearn.model_selection import GridSearchCV
# Define the hyperparameters you want to tune
params = {
    'hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,), (10,10,10), (20,20,20)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01, 0.1]
}

# Use GridSearchCV to search over the hyperparameter grid
grid_search = GridSearchCV(mlp, param_grid=params, cv=5, n_jobs=-1)

# Fit the GridSearchCV object on the training data
grid_search.fit(X_train, y_train)

# Print the best set of hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model on the validation set using the best hyperparameters
mlp = grid_search.best_estimator_

c:\Users\mmart\Programming\EECS545\project\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best Hyperparameters: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'learning_rate_init': 0.1}


In [56]:
# Evaluate the model
import numpy as np
from sklearn.metrics import mean_squared_error
predict_test = mlp.predict(X_test)
test_set_rsquared = mlp.score(X_test, y_test)
test_set_rmse = np.sqrt(mean_squared_error(predict_test, y_test))
print('R_squared value: ', test_set_rsquared) # Variances are not nicely correlated
print('RMSE: ', test_set_rmse) # Fairly good at data prediction


R_squared value:  0.5188831134787
RMSE:  4387398.389624616


In [57]:
# Save Model
import pickle
from common import NN_FILENAME
pickle.dump(mlp, open('saved_models/' + NN_FILENAME, 'wb'))